In [4]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import FeatureExtractor
from aldiscore.prediction import utils
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
from pathlib import Path
from Bio import SeqIO
from aldiscore import ROOT, get_from_config
import numpy as np
import pandas as pd

Trials:
- tc_base vs tc_different_statistics!
- tc_base vs tc_dist_abs vs tc_dist_scaled vs tc_dist_log
- tc_base + (tc_dist_abs vs tc_dist_scaled vs tc_dist_log)
- tc_base vs tc_50 vs tc_100

TODOs:
- Try new/better aggregation statistics
- Add remaining PSA features
- Look into simulation paper again


In [5]:
TEST_FILE_AA = Path("data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")
INTRON_FILE = Path("../../data/backup/alns/GALGAL_R00007_intron1.fa")

sequences = list(SeqIO.parse(TEST_FILE_AA, "fasta"))[:10]
for seq_record in sequences:
    seq_record.seq = Seq(str(seq_record.seq).replace("-", ""))

print("k =", len(sequences))

seq_lens = [len(seq) for seq in sequences]
print(seq_lens)
# print(sequences[0].seq[:100])
# print(sequences[29].seq[:100])
extractor = FeatureExtractor(sequences, track_perf=True)
extractor._init_cache()
feats = extractor.compute()
feats

# feats.loc[:, feats.columns.str.contains("chi-square")]
# feats.loc[:, feats.columns.str.contains("tc_")]

k = 4
[104, 120, 456, 540]


,is_dna,num_seqs,min:seq_length,max:seq_length,mean:seq_length,std:seq_length,p1:seq_length,p5:seq_length,p10:seq_length,p20:seq_length,...,p30:9mer_ent,p40:9mer_ent,p50:9mer_ent,p60:9mer_ent,p70:9mer_ent,p80:9mer_ent,p90:9mer_ent,p95:9mer_ent,p99:9mer_ent,iqr:9mer_ent
0,False,4,104,540,305.0,195.353526,104.48,106.4,108.8,113.6,...,6.785344,7.207566,7.80754,8.407514,8.832288,8.906661,8.981034,9.01822,9.04797,2.188032


In [6]:
counts, lens = utils.repeat_distributions(sequences)

print(counts.shape)

import seaborn as sn

sn.histplot(counts.T, element="poly")

(4, 43)


KeyboardInterrupt: 

In [ ]:
import pyfastx

for item in pyfastx.Fastx(TEST_FILE_DNA, uppercase=True):
    print(item)

('hmgl_trybr', 'AAGAAGGACTCTAATGCTCCGAAGCGGGCTATGACTTCTTTTATGTTCTTCTCATCGGACTTCCGAAGCAAGCATAGCGACTTATCGATAGTAGAGATGAGTAAGGCGGCAGGGGCAGCCTGGAAGGAGCTGGGGCCCGAGGAGCGCAAGGTGTATGAGGAAATGGCAGAGAAGGATAAAGAACGCTATAAGCGTGAGATG')
('hmgt_mouse', 'AAGCCTAAACGTCCTCGAAGTGCTTACAACATCTATGTCTCAGAAAGTTTTCAGGAGGCGAAAGATGACTCCGCGCAGGGCAAGTTGAAACTCGTTAATGAAGCCTGGAAAAATCTAAGTCCTGAGGAGAAGCAAGCCTATATCCAGTTGGCAAAAGACGACAGAATACGCTATGACAATGAAATGAAGTCATGGGAAGAGCAAATGGCTGAA')
('hmgb_chite', 'GCGGATAAACCTAAACGGCCATTAAGTGCGTACATGCTATGGCTAAATTCCGCCCGAGAATCTATTAAAAGAGAAAATCCTGATTTTAAAGTGACCGAGGTAGCTAAAAAGGGTGGTGAATTGTGGCGTGGTCTAAAGGATAAGTCGGAGTGGGAGGCAAAGGCTGCCACCGCTAAACAAAATTACATCAGGGCGCTTCAGGAATATGAGCGGAATGGCGGA')
('hmgl_wheat', 'GACCCTAATAAACCAAAACGCGCGCCCTCAGCTTTCTTCGTGTTTATGGGGGAGTTCAGAGAAGAATTTAAACAAAAAAATCCCAAGAACAAGTCCGTCGCCGCTGTTGGTAAGGCTGCGGGGGAACGATGGAAGAGTCTTTCAGAGAGCGAAAAGGCGCCGTACGTGGCCAAAGCAAATAAATTAAAAGGAGAGTATAATAAGGCGATTGCCGCATACAATAAGGGCGAATCTGCA')


In [ ]:
from itertools import groupby
from collections import defaultdict
import pytrf, pyfastx
from functools import partial

# from pytrf import d


def tandem_repeats(sequences):
    # parse input fasta sequence
    for seq_record in sequences:
        seq = str(seq_record.seq).upper()
        name = seq_record.id
        # get the first sequence from fasta
        # feed sequence to pytrf
        # the fastest way to get all SSRs from sequence
        ssrs = pytrf.STRFinder(name, seq).as_list()
        print(ssrs)


from aldiscore.prediction import utils

count_arr, len_arr = tandem_repeats(sequences)
len_arr

[('Acanthisitta_chloris', 509, 522, 'A', 1, 14, 14)]
[]
[]
[]
[]
[]


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
perf_dir = ROOT / "logs" / "perf"
perf_file = sorted(os.listdir(perf_dir))[-1]
print(perf_file)
perf_df = pd.read_parquet(perf_dir / perf_file)
perf_df.groupby("source").max().sum(axis=1)

2025-08-25_11-13-08.parquet


source
arthropod        4.868850
bali2dna         3.636999
bali2dnaf        3.722938
bali3            6.578528
ox               3.705403
prefab4          3.939462
sabre            0.912873
treebase_v1    591.011598
dtype: float64

In [ ]:
from tqdm import tqdm

source = "treebase_v1"
data_dir = Path("/hits/fast/cme/bodynems/data/paper") / source
datasets = list(
    filter(lambda name: os.path.isdir(data_dir / name), os.listdir(data_dir))
)

feats_dict = {}
for i, dataset in tqdm(enumerate(datasets)):
    # if dataset != "25479_0.phy":
    #     continue
    seqs = list(SeqIO.parse(data_dir / dataset / "sequences.fasta", format="fasta"))
    feats = FeatureExtractor(seqs).compute()
    feats_dict[(source, dataset)] = feats
    if i >= 24:
        break
feat_df = pd.concat(feats_dict.values(), axis=0, ignore_index=True)
feat_df.index = pd.MultiIndex.from_tuples(
    feats_dict.keys(), names=["source", "dataset"]
)

# feat_df.info()

In [ ]:
import parasail
from Bio.Seq import Seq
from Bio import SeqIO
import itertools
import random
from aldiscore.scoring.encoding import encode_positions
from aldiscore.enums.enums import PositionalEncodingEnum as PSE
from collections import defaultdict

seq_path = Path("/hits/fast/cme/bodynems/data/paper/prefab4/1cvl_1tca/sequences.fasta")
sequences = list(SeqIO.parse(seq_path, format="fasta"))
print(len(sequences))
print(sequences)
GAP_CHAR = "-"
# sequences = [
#     SeqRecord(Seq("ACCCA")),
#     SeqRecord(Seq("ACAGTAAACCA")),
#     SeqRecord(Seq("ACGGTACATTA")),
#     SeqRecord(Seq("TTCTTAG")),
#     SeqRecord(Seq("GCTACTATTA")),
#     SeqRecord(Seq("TTAATGGGGGGA")),
#     SeqRecord(Seq("ATTTCTA")),
#     SeqRecord(Seq("ATTTATTAAACCA")),
#     SeqRecord(Seq("ATTTGTTACATTA")),
#     SeqRecord(Seq("TTTTCTTAG")),
#     SeqRecord(Seq("GTTTTTCTATTA")),
#     SeqRecord(Seq("TTTTATTGGGGGGA")),
# ]
extractor = FeatureExtractor(sequences)
extractor._init_cache()
out = extractor.compute()
print(len(out.columns))
print(len(out.columns[out.columns.str.contains("tc_")]))
for i, col in enumerate(out.columns):
    print(col, end=" | ")
    if (i + 1) % 6 == 0:
        print()
# idx_map = extractor._cache[extractor._PSA_INDEX_MAP]
# for idx_a in idx_map:
#     print(idx_map[idx_a])

# print(groups)
# for idx_a in aligns:
#     print(idx_a)
#     for idx_b in aligns[idx_a]:
#         print("  " + str(idx_b))